In [ ]:
import torch
def update(q, k, v, m_i,acc_o, softmax_scale=1.0):
    qk = q @ k.transpose(-2, -1)*softmax_scale
    m_ij = torch.maximum(torch.max(qk, dim=-1, keepdim=True)[0], m_i)
    p = torch.exp(qk - m_ij)
    l_ij = torch.sum(p, dim=-1, keepdim=True)
    acc_o_scale = torch.exp(m_i-m_ij)
    acc_o = p @ v + acc_o_scale * acc_o
    return m_ij, l_ij, acc_o_scale, acc_o





def flash_attn(query, key, value):
    softmax_sclae = 1.0
    qs = torch.chunk(query, chunks=4, dim=2)
    ks = torch.chunk(key, chunks=4, dim=2)
    vs = torch.chunk(value, chunks=4, dim=2)
    q = qs[0]
    k = ks[0]
    v = vs[0]
    batch_size, num_heads, seqlen, head_dim = q.shape
    qk = q @ k.transpose(-2, -1)
    m_i = torch.max(qk, dim=-1, keepdim=True)[0]
    p = torch.exp(qk - m_i)
    l_ij = torch.sum(p, dim=-1, keepdim=True)
    lse_i = torch.log(l_ij) + m_i
    acc_o_scale = torch.exp(-m_i)
    acc_o = p @ v
    for i in range(3):
        k = ks[i + 1]
        v = vs[i + 1]
        m_ij, l_ij, acc_o_scale, acc_o = update(q, k, v, m_i,acc_o, softmax_sclae)
        m_i = m_ij
        l_i_new = torch.exp(lse_i - m_ij) + l_ij
        lse_i = m_ij + torch.log(l_i_new)
    o_scale = torch.exp(m_i - lse_i)
    acc_o = o_scale * acc_o
    return acc_o

    
def ref_attn(q, k, v):
    s = q @ k.transpose(-2, -1)
    s = torch.softmax(s, dim=-1)
    p = s @ v
    return p

batch = 2
seqlen = 1024
head_dim = 32
num_heads = 16
q = torch.randn((batch, num_heads, seqlen, head_dim),device="cuda")
k = torch.randn((batch, num_heads, seqlen, head_dim),device="cuda")
v = torch.randn((batch, num_heads, seqlen, head_dim),device="cuda")
p2 = ref_attn(q, k, v)
# print(torch.allclose(p1, p2))

In [85]:
from burst_attn_simple import OpBurstAttn




In [86]:
p1.shape

torch.Size([2, 16, 256, 32])

In [87]:
sub_p2[0].shape

torch.Size([2, 16, 256, 32])

In [88]:
(sub_p2[0] -p1).abs().max()

tensor(8.1062e-06, device='cuda:0')

In [58]:
p1[0,0,2]

tensor([-0.4493,  0.3916, -0.0198,  0.8010, -0.1278,  0.0292, -0.6342,  0.5712,
        -0.1580, -0.3107, -0.3507,  0.4695, -0.2947, -0.1443, -0.2678, -0.1860,
         0.0812,  0.0719,  0.1503,  0.2287, -0.4244,  0.9350,  0.3308, -0.5447,
         1.1858, -0.1079, -1.1268, -0.5063, -0.4900,  0.0082, -0.0855, -0.4401],
       device='cuda:0')